In [1]:
# import pandas lib
import pandas as pd

In [2]:
# vaiable for path to the file
path_to_file = '2_bookings.csv'

In [17]:
# import data frame
bookings = pd.read_csv(path_to_file, encoding='utf-8', sep=';')

- `Hotel` – тип отеля (City Hotel или Resort Hotel)  
- `Is canceled` – бронирование было отменено (1) или нет (0); не отмененное считается успешным
- `Lead time` – количество дней, прошедших между датой бронирования и датой прибытия  
- `Arrival full date` – полная дата прибытия
- `Arrival date year` – год прибытия  
- `Arrival date month` – месяц прибытия  
- `Arrival date week number` – номер недели прибытия
- `Arrival date day of month` – день прибытия
- `Stays in weekend nights` – количество выходных (суббота или воскресенье), которые гость забронировал для проживания в отеле
- `Stays in week nights` – количество дней (с понедельника по пятницу), которые гость забронировал для проживания в отеле
- `Stays total nights` – общее число забронированных ночей (сумма двух предыдущих колонок)
- `Adults` – число взрослых
- `Children` – число детей
- `Babies` – число младенцев 
- `Meal` – выбранный тип питания
- `Country` – страна происхождения клиента
- `Reserved room type` – тип зарезервированного номера
- `Assigned room type` – тип полученного номера (может отличаться от забронированного)
- `Customer type` – тип бронирования
- `Reservation status` – значение последнего статуса брони: Canceled - было отменено клиентом; Check-Out - клиент зарегистрировался, но уже покинул отель; No-Show - клиент не зарегистрировался и сообщил администрации отеля причину
- `Reservation status date` – дата обновления статуса

Let's make exploratory analysis

In [15]:
bookings.head()

,hotel,is_canceled,lead_time,arrival_full_date,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,adults,children,babies,meal,country,reserved_room_type,assigned_room_type,customer_type,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015-07-01,2015,July,27,1,0,0,...,2,0.0,0,BB,PRT,C,C,Transient,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,C,Transient,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015-07-01,2015,July,27,1,0,1,...,1,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015-07-01,2015,July,27,1,0,2,...,2,0.0,0,BB,GBR,A,A,Transient,Check-Out,2015-07-03


In [5]:
bookings.shape

(119390, 21)

In [6]:
bookings.dtypes

Hotel                         object
Is Canceled                    int64
Lead Time                      int64
arrival full date             object
Arrival Date Year              int64
Arrival Date Month            object
Arrival Date Week Number       int64
Arrival Date Day of Month      int64
Stays in Weekend nights        int64
Stays in week nights           int64
stays total nights             int64
Adults                         int64
Children                     float64
Babies                         int64
Meal                          object
Country                       object
Reserved Room Type            object
Assigned room type            object
customer type                 object
Reservation Status            object
Reservation status_date       object
dtype: object

### Task 1.
Write the first 7 lines to the bookings_head variable

In [7]:
bookings_head = bookings.head(7)

### Task 2.
Look at the dimensions of the table. How many columns are there in total?

In [10]:
bookings.shape[1]

21

### Task 3.
What type do most variables belong to?

In [13]:
bookings.dtypes.value_counts()

object     10
int64      10
float64     1
dtype: int64

### Task 4.
As you may have noticed, a small error occurred when collecting the dataset, and the names of the columns were recorded in an inconvenient format for further work. Correct the situation by reducing the column names to lowercase and replacing spaces with underscores.
For example: Is Cancelled -> is_canceled

In [19]:
bookings.columns = bookings.columns.str.lower().str.replace(' ','_')
bookings.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_full_date',
       'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'stays_total_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date'],
      dtype='object')

### Task 5.
Let's move on to data exploration! Users from which countries have made the largest number of successful bookings? The booking is considered successful if it has not been canceled in the future (the is_canceled variable). As an answer, select the countries in the top 5.

In [37]:
bookings.query("is_canceled == 0") \
    .country.value_counts() \
    .head()

PRT    21071
GBR     9676
FRA     8481
ESP     6391
DEU     6069
Name: country, dtype: int64

### Task 6.
For how many nights (stays_total_nights) do City Hotel type hotels book on average? Resort Hotel? Write down the received values in the gaps with an accuracy of 2 digits after the dot.

In [11]:
bookings \
    .groupby('hotel', as_index=False) \
    .agg({'stays_total_nights': 'mean'}) \
    .round(2)

,hotel,stays_total_nights
0,City Hotel,2.98
1,Resort Hotel,4.32


### Task 7.
Sometimes the type of room assigned to the client (assigned_room_type) differs from the originally booked one (reserved_room_type). This can happen, for example, due to overbooking.

How many similar observations were found in the dataset?
*cancellation of the reservation is also considered

In [38]:
bookings.query('assigned_room_type != reserved_room_type').hotel.count()

14917

### Task 8.
Now analyze the dates of the planned arrival (arrival_date_year).
For which month did you most often make a reservation in 2016? Has the most popular month changed in 2017?

In [45]:
bookings \
    .query('is_canceled == 1 & arrival_date_year == [2016, 2017]') \
    .groupby(['arrival_date_year','arrival_date_month'], as_index=False) \
    .agg({'hotel': 'count'}) \
    .rename(columns={'hotel':'num_of_reservations'}) \
    .sort_values('num_of_reservations', ascending=False) \
    .head(2)

,arrival_date_year,arrival_date_month,num_of_reservations
19,2017,May,2762
10,2016,October,2514


### Task 9.
Group the data by year, and then check for which month (arrival_date_month) City Hotel hotel bookings were canceled most often in 2015? 2016? 2017?

In [62]:
bookings \
    .query("is_canceled == 1 & hotel == 'City Hotel'") \
    .groupby('arrival_date_year') \
    .arrival_date_month.describe().top

arrival_date_year
2015    September
2016      October
2017          May
Name: top, dtype: object

### Task 10.
Look at the numerical characteristics of the three columns: adults, children and babies. Which of them has the highest average value?

In [57]:
bookings[['adults', 'children','babies']].describe().loc['mean']

adults      1.856403
children    0.103890
babies      0.007949
Name: mean, dtype: float64

### Task 11.
Create a total_kids column by combining the children and babies columns.
For which type of hotels did the average value of the variable turn out to be the largest?

- `City hotel` – the hotel is located in the city
- `Resort hotel` – resort hotel

As an answer, specify the largest average total_kids, rounded to 2 digits after the dot.

In [59]:
bookings['total_kids'] = bookings.children + bookings.babies

In [65]:
bookings[['hotel', 'total_kids']] \
    .groupby('hotel', as_index=False) \
    .agg({'total_kids': 'mean'}) \
    .round(2)

,hotel,total_kids
0,City Hotel,0.10
1,Resort Hotel,0.14


### Task 12.

Not all bookings were completed successfully (is_canceled), so you can calculate how many customers were lost in the process. In other words, calculate a metric called Churn Rate.

Churn rate (churn rate) is the percentage of subscribers (for example, to push notifications from the site) who unsubscribed from the communication channel, refused the services of the service for a certain period of time. In other words, it represents the ratio of the number of users who have left to the total number of users, expressed as a percentage.

In our case, the Churn Rate is the percentage of customers who have canceled a reservation. Let's see how this metric is related to the presence of children among customers!

Create the has_kids variable, which takes the value True if the customer specified at least one child (total_kids) when booking, otherwise False. Next, check among which group of users the churn rate is higher.

As an answer, specify the largest % of outflow, rounded to 2 digits after the dot (that is, the share of 0.24563 will be 24.56% and the answer will be 24.56)

In [66]:
bookings['has_kids'] = bookings.total_kids >= 1

In [67]:
bookings.has_kids.count()

119390

In [68]:
bookings.query("has_kids == True and is_canceled == 1").has_kids.count()

3259

In [69]:
#Count total and canceled Churn rate for families with kids and without them
crunch_rate_kids_true = bookings.query("has_kids == True and is_canceled == 1").has_kids.count()
crunch_rate_kids_false = bookings.query("has_kids == False and is_canceled == 1").has_kids.count()
crunch_rate_kids_true_total = bookings.query("has_kids == True").has_kids.count()
crunch_rate_kids_false_total = bookings.query("has_kids == False").has_kids.count()

In [72]:
# Churn rate with kids
(crunch_rate_kids_true / crunch_rate_kids_true_total * 100).round(2)

34.92

In [73]:
# Churn rate without kids
(crunch_rate_kids_false / crunch_rate_kids_false_total * 100).round(2)

37.22